In [1]:
using LinearAlgebra
using Optim
using HDF5

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [2]:
function infidelity_norm(ρ, σ)
    real(1 - tr(ρ * σ) - 2*sqrt(det(ρ)*det(σ)))
end

infidelity_norm (generic function with 1 method)

In [3]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

kossak_obj (generic function with 1 method)

In [4]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

initial_params .+= 1;

In [5]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [7]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"
γᶠ = parse(ComplexF64, γᵢ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γᵢ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γᵢ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γᵢ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γᵢ);

In [23]:
# Perform the optimization
result = optimize(objective, initial_params, 
                  method = BFGS(),
                  g_tol = 1e-10, show_trace = true;
                  autodiff = :forward)

Iter     Function value   Gradient norm 
     0     2.582814e+02     9.837154e+01
 * time: 0.017052888870239258
     1     4.868086e+01     1.863887e+01
 * time: 2.8085668087005615
     2     4.523568e+01     1.774879e+01
 * time: 4.832932949066162
     3     3.846488e+01     1.490910e+01
 * time: 6.95028281211853
     4     1.919964e+01     1.992261e+00
 * time: 10.14634084701538
     5     9.108636e+00     1.767067e+00
 * time: 13.316943883895874
     6     4.934070e+00     2.176612e+00
 * time: 15.428138017654419
     7     2.894144e+00     4.590449e+00
 * time: 18.614519834518433
     8     8.432554e-01     1.823794e+00
 * time: 21.786062955856323
     9     1.790269e-01     4.492820e-01
 * time: 24.925320863723755
    10     4.205312e-02     2.776398e-01
 * time: 28.079551935195923
    11     1.663042e-02     7.862424e-02
 * time: 31.223825931549072
    12     1.009264e-02     5.014804e-02
 * time: 34.37682485580444
    13     7.506160e-03     1.532990e-02
 * time: 37.481966972351

 * Status: success

 * Candidate solution
    Final objective value:     2.186071e-03

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 1.13e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.48e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.88e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 8.61e-12 ≰ 0.0e+00
    |g(x)|                 = 7.25e-11 ≤ 1.0e-10

 * Work counters
    Seconds run:   105  (vs limit Inf)
    Iterations:    35
    f(x) calls:    102
    ∇f(x) calls:   102


In [24]:
# Extract optimized parameters
optimized_params = result.minimizer

println("Optimized parameters: ", optimized_params)

Optimized parameters: [25.143185877022134, -0.0028407546411215886, 0.0004524071706840034, -0.19872217672070255, -0.20924294860078615, 0.004961691826453759, -0.19872206794978414, -0.20924304067820357, 0.004961696031060665, -0.19872206667474498, -0.20924303519837112, 0.0049616964479263954, -0.2426948917711709, 0.17562620865271034, -0.002496633844108799, -0.24269499614273365, 0.1756261150886797, -0.002496648073367302, -0.24269498983337115, 0.17562611390905444, -0.0024966475962700098]


In [25]:
# Substitute the optimized parameters back into H and C
optimized_H = construct_H(optimized_params)
optimized_C = construct_C(optimized_params)

println("Optimized H: ", optimized_H)
println("Optimized C: ", optimized_C)

Optimized H: ComplexF64[12.571592938511067 + 0.0im -0.0014203773205607943 + 0.0002262035853420017im; -0.0014203773205607943 - 0.0002262035853420017im -12.571592938511067 + 0.0im]
Optimized C: ComplexF64[0.29517395332909657 + 0.0im -0.003127108605797041 + 0.2570490615821529im -0.001140227130027367 - 0.005100949479076135im; -0.003127108605797041 - 0.2570490615821529im 0.22388153874410205 + 0.0im -0.004430027270210099 + 0.0010469946053581452im; -0.001140227130027367 + 0.005100949479076135im -0.004430027270210099 - 0.0010469946053581452im 9.255492630046557e-5 + 0.0im]


In [26]:
optimized_H

2×2 Matrix{ComplexF64}:
     12.5716+0.0im          -0.00142038+0.000226204im
 -0.00142038-0.000226204im     -12.5716+0.0im

In [11]:
optimized_H

2×2 Matrix{ComplexF64}:
     12.5716+0.0im          -0.00142037+0.000226205im
 -0.00142037-0.000226205im     -12.5716+0.0im

In [27]:
optimized_C

3×3 Matrix{ComplexF64}:
    0.295174+0.0im         -0.00312711+0.257049im    -0.00114023-0.00510095im
 -0.00312711-0.257049im       0.223882+0.0im         -0.00443003+0.00104699im
 -0.00114023+0.00510095im  -0.00443003-0.00104699im   9.25549e-5+0.0im

In [12]:
optimized_C

3×3 Matrix{ComplexF64}:
    0.295174+0.0im         -0.00312711+0.257049im    -0.00114022-0.00510094im
 -0.00312711-0.257049im       0.223882+0.0im         -0.00443002+0.00104699im
 -0.00114022+0.00510094im  -0.00443002-0.00104699im   9.25545e-5+0.0im

In [14]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end



get_lindblad_operators (generic function with 1 method)

In [15]:
Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
Cˢⁱᵈ = convert.(ComplexF64,optimized_C)

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

1-element Vector{Any}:
 ComplexF64[0.004810262472547129 + 0.0im -0.11367453194597005 - 0.49534483188090633im; -0.00484549139665192 - 0.03486923264058045im -0.004810262472547129 + 0.0im]

In [16]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [17]:
using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

In [18]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9995188642547398
Median fidelity for 0.25133: 0.9999049135197418
